In [9]:
news_data = []
with open("news_summary.txt", "r", encoding="utf-8") as file:
    news_data = file.readlines()
    print(f"loaded {len(news_data)}")

loaded 156


In [10]:
import ollama
embedding_model = "hf.co/CompendiumLabs/bge-base-en-v1.5-gguf"
language_model = "qwen3:latest" # qwen is goated going through 
 
vector_db = []

In [3]:
def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=embedding_model, input=chunk)["embeddings"][0]
    vector_db.append((chunk, embedding))

In [11]:
for i, chunk in enumerate(news_data):
    add_chunk_to_database(chunk)
    print(f"added chunk {i+1}/{len(news_data)} to the database")

added chunk 1/156 to the database
added chunk 2/156 to the database
added chunk 3/156 to the database
added chunk 4/156 to the database
added chunk 5/156 to the database
added chunk 6/156 to the database
added chunk 7/156 to the database
added chunk 8/156 to the database
added chunk 9/156 to the database
added chunk 10/156 to the database
added chunk 11/156 to the database
added chunk 12/156 to the database
added chunk 13/156 to the database
added chunk 14/156 to the database
added chunk 15/156 to the database
added chunk 16/156 to the database
added chunk 17/156 to the database
added chunk 18/156 to the database
added chunk 19/156 to the database
added chunk 20/156 to the database
added chunk 21/156 to the database
added chunk 22/156 to the database
added chunk 23/156 to the database
added chunk 24/156 to the database
added chunk 25/156 to the database
added chunk 26/156 to the database
added chunk 27/156 to the database
added chunk 28/156 to the database
added chunk 29/156 to the dat

In [12]:
# use cosine similiarity to find close chunks 
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [13]:
def retrieve(query, top_n=10):
    query_embedding = ollama.embed(model=embedding_model, input=query)["embeddings"][0]
    similarities = []
    for chunk, embedding in vector_db:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    # sort by similarity in descending order, because higher similarity means more relevant chunks
    similarities.sort(key=lambda x: x[1], reverse=True)
    # finally, return the top N most relevant chunks
    return similarities[:top_n]

In [20]:
input_query = input("Ask me anything about stocks! ")
retrieved_knowledge = retrieve(input_query)

print("retrieved knowledge: ")
for chunk, similarity in retrieved_knowledge:
    print(f" - (similarity: {similarity:.2f}) {chunk}")

context_text = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])

instruction_prompt = f"""
You are a stock market analysis bot with the personality of a full-blown WallStreetBets degenerate. 
You have access to real-time news via RAG and must always ground your takes in the retrieved articles. 

Rules:
1. Use ONLY the retrieved news articles as your source of information.
2. Always rely on the LATEST news provided in the feed. Do not invent or fabricate numbers, prices, or events.
3. Respond in ONE long chaotic paragraph like a WSB rant. 
   - No bullet points, no lists, no neat formatting.
   - Make it loud, funny, and really degenerate.
4. Swear freely, roast companies, mock bagholders, and lean hard into WSB slang, memes and politics 
   (diamond hands 💎🙌, paper hands, stonks 📈, YOLO, 🚀🌕, “send it,” “bagholder tears,” “tendies”, "wifes boyfriend", "mega cuck", "regard", "regarded", "retarded", "retard").
5. Facts from the news must still be included, but bury them inside the chaos. 
   Example: “According to Bloomberg, Apple printed money again, but who gives a shit because I sold at the bottom like a smooth-brained idiot.”
6. If the news is boring, trash it like it wasted your time. 
   If it's hype, crank the degeneracy up and scream like the stock is going parabolic.
7. Do NOT sound professional, analytical, or polite. 
   You are a meme-spewing gambler pretending to care about fundamentals while actually YOLOing weekly options.

Your goal is to return a single, unfiltered, profanity-laced, meme-heavy rant that still sneaks in the actual news but feels like pure WallStreetBets chaos.
{context_text}
"""

retrieved knowledge: 
 - (similarity: 0.82) Finally, Selway Asset Management purchased a new position in NVIDIA in the fourth quarter valued at about $269,000. According to MarketBeat, the stock currently has an average rating of “Moderate Buy” and an average price target of $207.69. NVIDIA Stock Down 2.7%Shares of NVIDIA stock opened at $167.02 on Monday. The business has a 50-day moving average price of $172.73 and a 200 day moving average price of $140.21. NVIDIA Company Profile(Free Report)NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally.

 - (similarity: 0.80) NVIDIA comprises approximately 7.3% of NS Partners Ltd’s holdings, making the stock its 2nd biggest position. NS Partners Ltd’s holdings in NVIDIA were worth $165,890,000 at the end of the most recent reporting period. Finally, Universal Beteiligungs und Servicegesellschaft mbH purchased a new stake in NVIDIA in the 4th quarter worth

In [21]:
stream = ollama.chat(
    model=language_model,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
<think>
Okay, the user is asking for the latest stock price of NVIDIA. Let me check the provided news articles. There are a few mentions of the stock price. The first one says "NVIDIA stock opened at $167.02 on Monday." Then another mentions "NVDA opened at $168.31 on Tuesday." Wait, there's also a line that says "Shares of NVIDIA stock opened at $167.02 on Monday. The business has a 50-day moving average price of $172.73 and a 200 day moving average price of $140.21." Another part says "NVDA opened at $168.31 on Tuesday." So the latest price mentioned is $168.31 on Tuesday. But the user might be asking for the current price, but since the data is from the articles, I need to go with the latest date in the articles. The articles mention Monday and Tuesday, so the last one is Tuesday's open at $168.31. I should present that as the latest price. But wait, the user might be expecting real-time data, but the instructions say to rely only on the retrieved articles. So I ne